# Baseline: bucket-of-chump Pre-optimized Solution

This establishes the baseline using the best publicly available pre-optimized solution.

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import json

getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(Decimal('0.0') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(Decimal('0.0') * scale_factor), float(base_y * scale_factor)),
            (float(-base_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(-mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(-top_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(-mid_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(-top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(-top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
        ])

        rotated_polygon = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated_polygon, float(self.center_x), float(self.center_y))

    @property
    def bounds(self):
        return self.polygon.bounds

def parse_value(val):
    """Parse value that may have 's' prefix"""
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def score_submission(csv_path):
    """Calculate the score for a submission CSV"""
    df = pd.read_csv(csv_path)
    
    per_n_scores = {}
    total_score = 0.0
    
    for n in range(1, 201):
        # Get rows for this N
        prefix = f'{n:03d}_'
        n_rows = df[df['id'].str.startswith(prefix)]
        
        if len(n_rows) != n:
            print(f'Warning: N={n} has {len(n_rows)} rows, expected {n}')
            continue
        
        # Create trees and compute bounding box
        trees = []
        for _, row in n_rows.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            trees.append(ChristmasTree(x, y, deg))
        
        # Compute bounding box
        all_bounds = [t.bounds for t in trees]
        min_x = min(b[0] for b in all_bounds)
        min_y = min(b[1] for b in all_bounds)
        max_x = max(b[2] for b in all_bounds)
        max_y = max(b[3] for b in all_bounds)
        
        side = max(max_x - min_x, max_y - min_y)
        score_n = (side ** 2) / n
        per_n_scores[n] = score_n
        total_score += score_n
    
    return total_score, per_n_scores

print('Scoring functions defined.')

In [ ]:
# Score the bucket-of-chump baseline solution
baseline_path = '/home/code/exploration/preoptimized/submission.csv'
baseline_score, per_n_scores = score_submission(baseline_path)

print(f'Baseline (bucket-of-chump) score: {baseline_score:.6f}')
print(f'Target score: 68.919154')
print(f'Gap to target: {baseline_score - 68.919154:.6f}')
print(f'Improvement needed: {(baseline_score - 68.919154) / baseline_score * 100:.2f}%')

In [ ]:
# Copy the baseline solution to submission folder
import shutil

# Copy to submission folder
shutil.copy(baseline_path, '/home/submission/submission.csv')
print('Copied baseline to /home/submission/submission.csv')

# Also copy to experiment folder for reference
shutil.copy(baseline_path, 'submission.csv')
print('Copied baseline to experiment folder')

In [ ]:
# Save metrics
metrics = {
    'cv_score': baseline_score,
    'target': 68.919154,
    'gap': baseline_score - 68.919154,
    'source': 'bucket-of-chump pre-optimized dataset'
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f'Saved metrics: {metrics}')